In [ ]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent))
from os import remove


from datetime import date, timedelta
import codecs
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

from os.path import join, basename

from bulletin import default_input, default_output, root, hoje, data_comeco_pandemia
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils import static, Timer
from bulletin.utils.xls_ import fit_cols

from datetime import datetime
import pyminizip as pz

today = datetime.today()
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)

municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']

import random

In [ ]:
download = True #and False ## <- comentar ´and False´ para baixar novo notifica
load_downloaded = True and False ## <- comentar ´and False´ para carregar partes baixadas
update = True and False ## <- comentar ´and False´ para atualizar notifica baixadas

if download:
    # Download
    mb = Metabase()
    mb.generate_notifica_query('notifica',where='True', replace=True)
    notifica_parts = mb.download_notificacao('notifica',load=load_downloaded)
    ##
    # Leitura
    notifica = Notifica('notifica')
    notifica.read(notifica_parts)
    notifica.save('all_notifica_raw',replace=True,compress=False)
    ##
    # Analise exploratória
    # analise_exploratoria(notifica)
    ##
    # Normalização
    notifica.normalize()
    notifica.save('notifica',replace=True, compress=False)
############
else:
    # Load 
    notifica = Notifica()
    notifica.databases()
    notifica.load('notifica',compress=False)
    
if update:
    mb = Metabase()
    days = 3
    intervalo = f"(data_notificacao >= NOW() - INTERVAL '{days} DAY') or (data_liberacao >= NOW() - INTERVAL '{days} DAY') or (updated_at >= NOW() - INTERVAL '{days} DAY') or (data_coleta >= NOW() - INTERVAL '{days} DAY') or (data_encerramento >= NOW() - INTERVAL '{days} DAY') or (data_cura_obito >= NOW() - INTERVAL '{days} DAY')"
    mb.generate_notifica_query('update_notifica', where=intervalo, replace=True)
    update_notifica_parts = mb.download_notificacao('update_notifica')
    update_notifica = Notifica()
    update_notifica.read(update_notifica_parts)
    update_notifica.normalize()
    update_notifica.save(f"update_notifica_{today.strftime('%d%m%Y%H%M')}")
    
#     notifica.check_update(update_notifica)
    notifica.update(update_notifica)
    notifica.fix_dtypes()
    notifica.save(replace=True, compress=False)
    notifica.df

In [ ]:
# Carrega casos confirmados sem merge
cc = CasosConfirmados()
cc.load(f"cca_{hoje.strftime('%d_%m_%Y')}",compress=False)

# Corrige colunas com base no informaction_schema
cc.df = cc.df.rename(columns={'exame':'exame_nome','data_evolucao':'data_cura_obito'})

# Zera casos encontrados
cc.df['id_notifica'] = -1
cc.df['acc_merge_notifica'] = 'NAO_ENCONTRADO'
cc.df['tipo_encerramento'] = 'E0'

# Seleciona colunas para concatenar com o id notifica
notifica_cols = cc.schema.loc[cc.schema['notifica']==1,'column']
novas_colunas = [ col for col in notifica_cols if not col in cc.df.columns ]
cc.df[novas_colunas] = None

# Corrige os tipos das colunas
cc.normalize()
cc.df

In [ ]:
cc.df.dtypes

In [ ]:

print(f"ids in casos {notifica.df['id'].isin(cc.df['id_notifica']).sum()}\n")
display(cc.df[['identificacao','acc_merge_notifica']].groupby('acc_merge_notifica').count().sort_values('identificacao',ascending=False))
print('------------------------------------------------------------------------')
# Percorre excluir ficha, classificacao final e status notificacao e cruza com os hashes com preferencia nas ordens percorridas
for excluir_ficha in [2,1]:
    for classificacao_final in [2,1,3,5,0]:
        for status_notificacao in [1,2,3,4]:
            casosn = notifica.df.loc[(notifica.df['classificacao_final']==classificacao_final)&(notifica.df['status_notificacao']==status_notificacao)&(notifica.df['excluir_ficha']==excluir_ficha)]
            print(f"notifica[classificacao_final] == {classificacao_final} & notifica[status_notificacao] == {status_notificacao} & notifica[excluir_ficha] == {excluir_ficha}: {len(casosn)}\n")
            if len(casosn) != 0:
                for notifica_hash_col, cc_hash_col in zip(['hash_resid','hash_resid','hash_resid','hash_atend','hash_atend','hash_atend','hash_lib'],['hash_resid','hash_resid_less', 'hash_resid_more', 'hash_atend', 'hash_atend_less','hash_atend_more', 'hash_diag']):
                    print(f"casosn[{notifica_hash_col}] <-> cc.df[{cc_hash_col}]")

                    # Casos notifica com hash não duplicado ou nulo. Apenas com as colunas usadas no merge 
                    casosn = casosn.loc[(~casosn.duplicated(notifica_hash_col,keep=False)) & (casosn[notifica_hash_col].notna())]
                    casosn_id = casosn[['id',notifica_hash_col]+novas_colunas].rename(columns={notifica_hash_col:cc_hash_col, 'id':'id_notifica'})
                    
                    # Casos confirmados sem id_notifica, com hash não duplicado ou nulo
                    casosc_hashed = cc.df.loc[((cc.df['id_notifica']==-1)|(cc.df['id_notifica'].isna()))&(cc.df[cc_hash_col].notna())]
                    casosc_hashed = casosc_hashed.loc[(~casosc_hashed.duplicated(cc_hash_col,keep=False))]

                    # Realiza merge com hash, deixando apenas o identificacao dos casos confirmados e as colunas do notifica
                    casosc_hashed = pd.merge(casosc_hashed[['identificacao',cc_hash_col]],casosn_id,on=cc_hash_col,how='left')
                    casosc_hashed = casosc_hashed.loc[(casosc_hashed['id_notifica']!=-1)&(casosc_hashed['id_notifica'].notna())]
                    casosc_hashed['acc_merge_notifica'] = f"{excluir_ficha}{classificacao_final}{status_notificacao}{cc_hash_col.upper()}"
                    print(f"{excluir_ficha}{classificacao_final}{status_notificacao}{cc_hash_col.upper()}")
                    casosc_hashed = casosc_hashed.set_index('identificacao')
                    
                    # Seta identificacao como index nos casos confirmados e realiza update com as colunas do notifica
                    cc.df.set_index('identificacao',inplace=True)
                    cc.df.update(casosc_hashed)
                    cc.df.reset_index(inplace=True)

                    print(f"ids in casos {notifica.df['id'].isin(cc.df['id_notifica']).sum()}\n")
                    display(cc.df[['identificacao','acc_merge_notifica']].groupby('acc_merge_notifica').count().sort_values('identificacao',ascending=False))
                    print('------------------------------------------------------------------------')

In [ ]:
cc.df[['identificacao','acc_merge_notifica']].groupby('acc_merge_notifica').count().sort_values('identificacao',ascending=False)

In [ ]:
cc.df.loc[cc.df['id_notifica']==-1,['identificacao','acc_merge_notifica']].groupby('acc_merge_notifica').count().sort_values('identificacao',ascending=False)

In [ ]:
cc.df.loc[cc.df['id_notifica']==-1,'id_notifica'] = [x for x in range(-1,(cc.df['id_notifica']==-1).sum()*-1,-1)]
cc.df

In [ ]:
cc.fix_dtypes()

In [ ]:
cc.df

In [ ]:
cc.save(f"ccan_{hoje.strftime('%d_%m_%Y')}",replace=True,compress=False)

In [ ]:
cc = CasosConfirmados()
cc.load(f"ccan_{hoje.strftime('%d_%m_%Y')}",compress=False)
casosc = cc.df
casosc